In [20]:
import tensorflow as tf
import numpy as np
import uuid
from nltk.tokenize import sent_tokenize
from gensim.models import KeyedVectors
import pickle
import itertools
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../../MNIST_data', one_hot=True)
batch_size = 200
original_dim = 1000
latent_dim = 100
intermediate_dim = 500
epochs = 100
epsilon_std = 1.0
tf.reset_default_graph()
def samp(z_mean,z_log_var):
    epsilon = tf.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + tf.exp(z_log_var / 2) * epsilon

def total_loss(x, y,z_mean,z_log_var):
    ent_loss =   tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(labels=x, logits=y),1)
    
    kl_loss = - 0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var),1)
    print(kl_loss.get_shape())
    #recon_loss = tf.reduce_sum(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=X), 1)
    # D_KL(Q(z|X) || P(z)); calculate in closed form as both dist. are Gaussian
    #kl_loss = 0.5 * tf.reduce_sum(tf.exp(z_logvar) + z_mu**2 - 1. - z_logvar, 1)
    return [ent_loss,kl_loss]

def split_into_sent (text):
    strg = ''
    for word in text:
        strg += word
        strg += ' '
    strg_cleaned = strg.lower()
    for x in ['\n','"',"!", '#','$','%','&','(',')','*','+',',','-','/',':',';','<','=','>','?','@','[','^',']','_','`','{','|','}','~','\t']:
        strg_cleaned = strg_cleaned.replace(x, '')
    sentences = sent_tokenize(strg_cleaned)
    return sentences

def vectorize_sentences(sentences):
    vectorized = []
    for sentence in sentences:
        byword = sentence.split()
        concat_vector = []
        for word in byword:
            try:
                concat_vector.append(w2v[word])
            except:
                pass
        vectorized.append(concat_vector)
    return vectorized

def batch_generator(sources,batch_size=50):
    idxs = np.random.permutation(np.arange(len(sources)))
    n_batches = len(idxs) // batch_size
    for batch_i in range(n_batches):
        this_idxs = idxs[batch_i * batch_size:(batch_i + 1) * batch_size]
        this_sources = sources[this_idxs, :]
        yield (this_sources)

def sent_parse(sentence, mat_shape):
    data_concat = []
    word_vecs = vectorize_sentences(sentence)
    print("wordvec length:",len(word_vecs))
    for x in word_vecs:
        data_concat.append(list(itertools.chain.from_iterable(x)))
    zero_matr = np.zeros(mat_shape)
    print(mat_shape)
    print(len(data_concat))
    zero_matr[0] = np.array(data_concat)
    return zero_matr


def print_sentence_with_w2v(sent_vect):
    word_sent = ''
    tocut = sent_vect
    for i in range (int(len(sent_vect)/100)):
        word_sent += w2v.most_similar(positive=[tocut[:100]], topn=1)[0][0]
        word_sent += ' '
        tocut = tocut[100:]
    print(word_sent)
    

def find_similar_encoding(sent_vect):
    all_cosine = []
    for sent in sent_encoded:
        result = 1 - spatial.distance.cosine(sent_vect, sent)
        all_cosine.append(result)
    data_array = np.array(all_cosine)
    maximum = data_array.argsort()[-3:][::-1][1]
    new_vec = sent_encoded[maximum]
    return new_vec


def interpolate_b_points(point_one, point_two, num):
    dist_vec = point_two - point_one
    sample = np.linspace(0, 1, num, endpoint = True)
    hom_sample = []
    for s in sample:
        hom_sample.append(point_one + s * dist_vec)
    return hom_sample


def sent_2_sent(sess,sent1,sent2, batch, dim,z_mean,x_decoded):
    a = sent_parse([sent1], (batch,dim))
   
    b = sent_parse([sent2], (batch,dim))
    encode_a = sess.run(z_mean,feed_dict={x:a})
    
    encode_b = sess.run(z_mean,feed_dict={x:b})
    intermediate_points = interpolate_b_points(encode_a[0], encode_b[0], 5)
    
    for point in intermediate_points:
        
        zero_matr = np.zeros((batch,latent_dim))
        zero_matr[0] = np.array(point)
        p = sess.run(x_decoded,feed_dict={z:zero_matr})[0]
        print_sentence_with_w2v(p)

def super_linear(x,
                     output_size,
                     scope=None,
                     reuse=tf.AUTO_REUSE,
                     init_w='gaussian',
                     weight_start=0.0,
                     use_bias=True,
                     bias_start=0.0,
                     input_size=None):
        """Performs linear operation. Uses ortho init defined earlier."""
        shape = x.get_shape().as_list()
        with tf.variable_scope(scope or 'linear'):
            if reuse is True:
                tf.get_variable_scope().reuse_variables()

            w_init = None  # uniform
            if input_size is None:
                x_size = shape[1]
            else:
                x_size = input_size
            if init_w == 'zeros':
                w_init = tf.constant_initializer(0.0)
            elif init_w == 'constant':
                w_init = tf.constant_initializer(weight_start)
            elif init_w == 'gaussian':
                w_init = tf.random_normal_initializer(stddev=weight_start)
            elif init_w == 'ortho':
                w_init = lstm_ortho_initializer(1.0)

            w = tf.get_variable(
            'super_linear_w', [x_size, output_size], tf.float32, initializer=w_init)
            if use_bias:
                b = tf.get_variable(
              'super_linear_b', [output_size],
              tf.float32,
              initializer=tf.constant_initializer(bias_start))
                return tf.matmul(x, w) + b
        return tf.matmul(x, w)
    

Extracting ../../MNIST_data\train-images-idx3-ubyte.gz
Extracting ../../MNIST_data\train-labels-idx1-ubyte.gz
Extracting ../../MNIST_data\t10k-images-idx3-ubyte.gz
Extracting ../../MNIST_data\t10k-labels-idx1-ubyte.gz


In [18]:
w2v = KeyedVectors.load_word2vec_format('apt_vectors.vec')
data_concat = []


with open('APT_sanitized.txt',"r",encoding='utf-8') as f:
    text=f.readlines()
vect = vectorize_sentences(text)
data = [x for x in vect  if len(x) == 10]
for x in data:
    data_concat.append(list(itertools.chain.from_iterable(x)))
    

data_array = np.array(data_concat)
np.random.shuffle(data_array)

train = data_array[:60000]
test = data_array[60000:80000]


In [24]:
tf.reset_default_graph()
x = tf.placeholder(shape=[batch_size, original_dim], dtype=tf.float32)
h = tf.layers.dense(x,intermediate_dim , activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer())
#h = tf.nn.relu(super_linear(x,intermediate_dim,scope="encoderhidden"))
z_mean=tf.layers.dense(h,latent_dim,name="encoder",kernel_initializer=tf.contrib.layers.xavier_initializer())
#z_mean = super_linear(h,latent_dim,scope="encodermean")
z_log_var =tf.layers.dense(h,latent_dim,kernel_initializer=tf.contrib.layers.xavier_initializer())
#z_log_var = super_linear(h,latent_dim,scope="encodervar")
z=samp(z_mean,z_log_var)
                             
h_decoded = tf.layers.dense(z,intermediate_dim, activation=tf.nn.relu,kernel_initializer=tf.contrib.layers.xavier_initializer())
#h_decoded = tf.nn.relu(super_linear(z,intermediate_dim,scope="decoderhidden"))
#x_decoded = tf.nn.sigmoid(super_linear(h_decoded,original_dim,"decoderrecon"))
x_decoded = tf.layers.dense(h_decoded,original_dim, activation=tf.nn.sigmoid,name="decoder",kernel_initializer=tf.contrib.layers.xavier_initializer())
print(x.get_shape())
print(x_decoded.get_shape())


xent_loss,kl_loss=total_loss(x,x_decoded,z_mean,z_log_var)
cost = tf.reduce_mean(xent_loss + kl_loss)
optimizer = tf.train.AdamOptimizer(1e-3).minimize(cost)

init=tf.global_variables_initializer()
coord = tf.train.Coordinator()

epoch_i=0
cycles=0
with tf.Session() as sess:
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    sess.run(init)
    try:
        while not coord.should_stop():
            for it_i, (this_sources) in enumerate(batch_generator(train,batch_size)):
            
            #X_mb, _ = mnist.train.next_batch(batch_size)
                _,l,s,kl = sess.run([optimizer,cost,xent_loss,kl_loss],feed_dict={x:this_sources})
                
                
                if cycles == 100:
                    coord.request_stop()
                if(epoch_i % 1000) == 0:
                    print("Epoch:",epoch_i)                    
                    print("Loss ",l)
                    sent_2_sent(sess,'already legitimately used in an Internet Explorer plugin open source','depending on Windows version . Returns failure or success with',batch=200,dim=1000,z_mean=z_mean,x_decoded=x_decoded)
                    cycles=cycles+1
                epoch_i=epoch_i+1
    except tf.errors.OutOfRangeError:
            print('Done training -- epoch limit reached')
    finally:
            # One of the threads has issued an exception.  So let's tell all the
            # threads to shutdown.
            coord.request_stop()

        # Wait until all threads have finished.
    coord.join(threads)
    #sent_2_sent(sess,'already legitimately used in an Internet Explorer plugin open source','depending on Windows version . Returns failure or success with',batch=200,dim=1000,z_mean=z_mean,x_decoded=x_decoded)

(200, 1000)
(200, 1000)
(200,)
Epoch: 0
Loss  1061.3225
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
pilfered utilise pilfered pilfered pilfered utilise utilise attack3 pilfered utilise 
pilfered utilise pilfered pilfered pilfered utilise utilise attack3 pilfered utilise 
pilfered utilise pilfered pilfered utilise utilise utilise attack3 pilfered utilise 
attack3 attack3 pilfered pilfered utilise pilfered utilise attack3 pilfered utilise 
attack3 attack3 pilfered pilfered utilise pilfered utilise utilise pilfered pilfered 
Epoch: 1000
Loss  666.0886
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing 
webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing 
webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing 
webfacing webfacing webfacing webfacing webfacing webfacin

Epoch: 12000
Loss  654.05255
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
available Backdoor.Agent.Hza available Backdoor.Agent.Hza infected webfacing webfacing webfacing webfacing webfacing 
webfacing Backdoor.Agent.Hza D3C Backdoor.Agent.Hza infected webfacing webfacing webfacing webfacing webfacing 
webfacing Backdoor.Agent.Hza klient Backdoor.Agent.Hza kyaukphyusez webfacing Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing webfacing Backdoor.Agent.Hza webfacing webfacing Backdoor.Agent.Hza D3C Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza infected Backdoor.Agent.Hza webfacing 
Epoch: 13000
Loss  657.61053
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
available Backdoor.Agent.Hza D3C Backdoor.Agent.Hza infected webfacing webfacing webfacing webfacing webfacing 
webfacing Backdoor.Agent.Hza D3C Backdoor.Agent.Hza infecte

Epoch: 24000
Loss  655.78546
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
D3C webfacing D3C Backdoor.Agent.Hza kyaukphyusez webfacing webfacing Backdoor.Agent.Hza webfacing webfacing 
webfacing webfacing D3C Backdoor.Agent.Hza webfacing webfacing webfacing Backdoor.Agent.Hza Backdoor.Agent.Hza webfacing 
webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing webfacing webfacing webfacing webfacing webfacing available Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing webfacing webfacing webfacing kyaukphyusez Backdoor.Agent.Hza webfacing 
Epoch: 25000
Loss  657.0514
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
specific Backdoor.Agent.Hza D3C Backdoor.Agent.Hza kyaukphyusez Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing webfacing 
webfacing webfacing provided Backdoor.Agent.Hza webfacing webfacing webfacing Backdoor.Age

Epoch: 37000
Loss  649.8644
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
D3C Backdoor­FCQ webfacing webfacing D3C webfacing webfacing Backdoor­FCQ webfacing common 
Cisco3045EK9 webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing 
webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing Backdoor­FCQ webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing webfacing webfacing webfacing infected Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing Backdoor­FCQ webfacing webfacing kyaukphyusez Backdoor.Agent.Hza webfacing 
Epoch: 38000
Loss  655.39264
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
D3C Backdoor.Agent.Hza webfacing webfacing D3C webfacing webfacing Backdoor.Agent.Hza webfacing common 
webfacing Backdoor­FCQ webfacing webfacing webfacing webfacing webfacing webfacing Backdoor­FCQ webfacing 
webfacing webfacing Backdoor­FCQ webfacing 

Epoch: 50000
Loss  652.5928
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
D3C Backdoor­FCQ webfacing webfacing D3C webfacing webfacing Backdoor.Agent.Hza webfacing webfacing 
Webbased webfacing webfacing webfacing Cisco3045EK9 webfacing webfacing webfacing Backdoor­FCQ webfacing 
webfacing webfacing Backdoor.Agent.Hza webfacing Cisco3045EK9 webfacing webfacing Cisco3045EK9 Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing webfacing webfacing webfacing infected Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing webfacing webfacing webfacing kyaukphyusez Backdoor.Agent.Hza webfacing 
Epoch: 51000
Loss  653.7399
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
kyaukphyusez Backdoor­FCQ webfacing webfacing D3C webfacing webfacing Backdoor.Agent.Hza webfacing common 
common webfacing webfacing webfacing Cisco3045EK9 webfacing webfacing webfacing Backdoor.Agent.Hza webfacing 
webfacing

Epoch: 62000
Loss  653.1938
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
D3C Backdoor­FCQ webfacing webfacing common webfacing webfacing Backdoor.Agent.Hza webfacing three 
specific webfacing webfacing webfacing common webfacing webfacing Backdoor.Agent.Hza Backdoor­FCQ webfacing 
webfacing webfacing webfacing webfacing webfacing webfacing webfacing webfacing Backdoor­FCQ webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing webfacing webfacing webfacing D3C Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza kyaukphyusez Backdoor.Agent.Hza webfacing 
Epoch: 63000
Loss  655.377
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
Themes Backdoor.Agent.Hza webfacing webfacing common webfacing webfacing Backdoor.Agent.Hza webfacing available 
spray webfacing webfacing webfacing spray webfacing webfacing Backdoor.Agent.Hza Backdoor.Agent.Hza webfacing 
webfacing

Epoch: 74000
Loss  654.6156
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
kyaukphyusez Backdoor­FCQ webfacing webfacing available webfacing webfacing Backdoor.Agent.Hza webfacing three 
infected webfacing webfacing webfacing Cisco3045EK9 webfacing webfacing Backdoor.Agent.Hza Backdoor.Agent.Hza webfacing 
webfacing Cisco3045EK9 Backdoor.Agent.Hza webfacing Cisco3045EK9 webfacing webfacing webfacing Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza spray Backdoor.Agent.Hza webfacing Cisco3045EK9 webfacing webfacing Webbased Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza infected Backdoor.Agent.Hza webfacing 
Epoch: 75000
Loss  654.0814
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
infected Backdoor.Agent.Hza webfacing webfacing additional webfacing webfacing Backdoor.Agent.Hza Backdoor.Agent.Hza webfacing 
spray webfacing webfacing webfacing spray webfacing webfaci

Epoch: 86000
Loss  651.9321
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
D3C webfacing webfacing webfacing common webfacing webfacing Backdoor.Agent.Hza webfacing three 
three webfacing webfacing webfacing spray illustrate illustrate Backdoor.Agent.Hza Backdoor.Agent.Hza webfacing 
webfacing webfacing Backdoor.Agent.Hza webfacing webfacing illustrate illustrate webfacing Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing webfacing illustrate webfacing Themes Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza kyaukphyusez Backdoor.Agent.Hza webfacing 
Epoch: 87000
Loss  654.3714
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
D3C webfacing webfacing webfacing common webfacing webfacing Backdoor.Agent.Hza webfacing three 
specific webfacing webfacing webfacing spray webfacing illustrate webfacing Backdoor.Agent.Hza webfacing 
webfaci

Epoch: 98000
Loss  656.57245
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
kyaukphyusez webfacing webfacing webfacing available webfacing webfacing Backdoor.Agent.Hza webfacing available 
infected Cisco3045EK9 webfacing webfacing spray illustrate illustrate Backdoor.Agent.Hza Backdoor.Agent.Hza webfacing 
spray spray Backdoor.Agent.Hza webfacing webfacing illustrate illustrate webfacing Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza spray Backdoor.Agent.Hza webfacing webfacing illustrate Backdoor.Agent.Hza Themes Backdoor.Agent.Hza webfacing 
Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza webfacing Backdoor.Agent.Hza infected Backdoor.Agent.Hza webfacing 
Epoch: 99000
Loss  654.9413
wordvec length: 1
(200, 1000)
1
wordvec length: 1
(200, 1000)
1
kyaukphyusez klient webfacing webfacing available webfacing webfacing Backdoor.Agent.Hza webfacing three 
infected webfacing webfacing webfacing spray webfacing illustrate Backdoor.Agent.Hza Bac